DEPRECATED

In [ ]:
# Automatic reloading of modules during development
%load_ext autoreload
%autoreload 2

# Create some folders
if not os.path.isdir("figures"): 
    os.mkdir("figures")
if not os.path.isdir("cache"):
    os.mkdir("cache")
    
# Monkeypatch some default values
sfg2d.core.VIS_WL = 812.7
sfg2d.core.PUMP_FREQ = 2400
#sfg2d.core.BASE_SPEC = Array_of_data
#sfg2d.core.NORM_SPEC = Array_of_data

# Reload cached results 
#sample = sfg2d.SfgRecord("cache/")

## Crosscorrelation

In [ ]:
cc0 = sfg2d.SfgRecord("./") # Fill in path

sfg2d.replace_pixel(cc0, 496)
cc0.base = bg0 # Use correct baseline
cc0.roi_x_pixel_spec = slice(500, 1300) # x-pixel slice of interest
cc0.rois_x_pixel_trace = [cc0.roi_x_pixel_spec]
cc0.roi_spectra = [0] # Only look at the first spectrum

xdata = cc0.pp_delays[cc0.roi_delay]
ydata = cc0.traces_basesubed.squeeze()

ydata = ydata/ydata.max()
gm = sfg2d.models.GaussianModel(xdata, ydata, p0=[400, 0, 180, 0])
gm.curve_fit()
gm.box_coords = (.1, .3)


# Pump SFG

In [ ]:
# Pump-Sfg and Probe-SFG
pump = sfg2d.SfgRecord("")

sfg2d.replace_pixel(pump, 496)
pump.base = bg0 # Set correct baseline
pump.norm = norm_spec
pump.rois_x_pixel_trace=[slice(600, 800)]
pump.roi_x_pixel_spec=pump.rois_x_pixel_trace[0]
pump.roi_spectra=[0]
pump.save("cache/pump")

xdata, ydata = pump.subselect(y_property='basesubed', x_property="wavenumber", delay_mean=True)
ydata =ydata.squeeze()

ydata = ydata/ydata.max()
gm = sfg2d.models.GaussianModel(xdata, ydata, p0=[1, 2500, 100, 0])
gm.curve_fit()
gm.box_coords = (.3, .1)

In [ ]:
fig, ax = subplots()
sfg2d.plotting.spec_plot(
    pump, 
    title="Pump SFG",
    ylabel="Counts in {}ms".format(pump.exposure_time_ms)
)
sfg2d.savefig("figures/pump")

fig, ax = subplots()
sfg2d.plotting.spec_plot(
    pump, 
    title="Pump SFG",
    ylabel="Counts in {}ms".format(pump.exposure_time_ms),
    delay_mean=True,
)
sfg2d.savefig("figures/pump_mean")

fig, ax = subplots()
gm.plot(show_fit_line=True, show_box=True)
sfg2d.savefig('figures/pump_mean_fit')

fig, ax = subplots()
sfg2d.plotting.trace_plot(
    pump,
    title="Pump-Vis Trace"
)
sfg2d.savefig("figures/pump_trace")

# HEAT

In [ ]:
hot = 
cold = 

fig, ax = subplots(2, 2, figsize=(9, 8))
sca(ax[0, 0])
title("Raw Static Spectrum")
cold.plot_spec(y_property="basesubed", x_property="wavenumber", label = "Cool")
hot.plot_spec(y_property="basesubed", x_property="wavenumber", label = "Hot")
xlabel("wavenumber in 1/cm")
legend()

sca(ax[0, 1])
title("Normalized Spectrum")
cold.plot_spec(y_property="normalized", x_property="wavenumber", medfilt_kernel=(5), marker = "o", label = "Cool")
hot.plot_spec(y_property="normalized", x_property="wavenumber", medfilt_kernel=(5), marker = "o", label = "Hot")
xlabel("wavenumber in 1/cm")
ylim(0.002, 0.012)
legend()

sca(ax[1, 0])
bleach = sfg2d.core.medfilt(median(hot.basesubed / cold.basesubed, 1)[0, 1], 5)
plot(cold.wavenumber, bleach)
title("Heat Bleach Rel")
xlim(2100, 2800)
ylim(1, 1.4)

sca(ax[1,1])
bleach = sfg2d.core.medfilt(median(hot.normalized - cold.normalized, 1)[0, 1], 5)
plot(cold.wavenumber, bleach)
title("Heat Bleach Abs")
xlim(2100, 2800)
ylim(0, 0.0025)
tight_layout()
sfg2d.savefig("figures/{}_heat_bleach".format(hot.metadata["material"]))

# Pump Probe

loading of pump probe

In [ ]:
g0 = sfg2d.SfgRecord("./")# Fill in path
record = g0

# Victor Camera has a broken Pixel 496. This replaces the values of the broken pixel.
sfg2d.replace_pixel(record, 496)
record.base = bg0 # Use correct baseline
record.norm = norm_spec
record.pumped_index = 2
record.unpumped_index = 0
record.rois_x_pixel_trace=[slice(600, 800),] # Where the pump is and you want to have traces.
record.roi_x_pixel_spec = slice(200, 1400) # Where the static signal is
record.zero_time_selec=[0, 1, 2] # Lets use the first the pp-delay points for offset correction
record.zero_time_subtraction = True # Perform zero time offset correction.
record.save("cache/{}_pump{}".format(record.metadata['material'], record.metadata['pump_freq']))

#record.save("cache/gold")

# Bleach 

In [ ]:
record = g0 # Use correct SfgRecord.
attribute = "bleach_abs"
attribute = "bleach_rel"

@sfg2d.multipage_pdf # Within the decorator we hide the loop
def my_plot(index):

    record.plot_bleach(
        y_property=attribute, 
        roi_delay=[slice(index, index+1)],
        x_property="wavenumber"
    )

    title("{} @ {} fs".format(record.metadata.get("sample"), record.pp_delays[index]))
    # Usually the limits need some tweaking
    #xlim(2000, 3000) 
    ylim(0.4, 1.1)
    xlabel("wavenumber in 1/cm")

name = "figures/{}_{}_pump{}".format(
    record.metadata.get("sample"), attribute, record.metadata.get("pump_freq")
)
my_plot(name, range(record.number_of_pp_delays))

# 2D Spectrum

In [ ]:

record = 

#record.rois_x_wavenumber_trace = [slice(2309, 2449), slice(2460, 2568), slice(2578, 2661), slice(2686, 2793)]
levels = linspace(0.87, 1, 15) 
fig, ax, axl, axb, axr =sfg2d.plotting.contour(
    record, 
    pixel_med=11,
    N=50,
    levels=levels,
    show_colorbar=False,
    show_axl=True
)
axl.set_xlim(0.6, 1.1)
axb.set_xlim(-1000, 5000)

name = "figures/{}_img_pump{}".format(
    record.metadata.get("material"), record.metadata.get("pump_freq"), 
)
sfg2d.savefig(name, dpi=80)